In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import glob
import os
from datetime import datetime
from openpyxl import Workbook
from datetime import datetime

# WebDriver 사용을 위한 라이브러리
from selenium import webdriver

# Requests 및 HTTPAdapter 사용을 위한 라이브러리
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# BeautifulSoup 사용을 위한 라이브러리 (BS4 setting에 사용된다고 언급됨)
from bs4 import BeautifulSoup

In [2]:
# url
url = 'https://m.place.naver.com/restaurant/1085956231/review/visitor?entry=ple&reviewSort=recent'

# Webdriver headless mode setting
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# BS4 setting for secondary access
session = requests.Session()
headers = {
    #사용자의 user-agent값
    "User-Agent": "$User value"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[500, 502, 503, 504])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [9]:
from datetime import datetime

# New xlsx file
now = datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [11]:
!pip install webdriver-manager

In [13]:
# Start crawling/scraping!
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

res = driver.get(url)
driver.implicitly_wait(30)

# Pagedown
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

try:
    while True:
        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
        time.sleep(0.4)
except Exception as e:
    print('finish')

finish


In [15]:
time.sleep(25)
html = driver.page_source
bs = BeautifulSoup(html, 'lxml')
reviews = bs.select('li.pui__X35jYm.EjjAW')

for r in reviews:
    # nickname
    nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

    # content
    content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

    # date
    date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
    date = date_elements[0] if date_elements else 'N/A'

    # revisit
    revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
    revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

    # exception handling
    nickname = nickname.text if nickname else ''
    content = content.text if content else ''
    date = date.text if date else ''
    revisit = revisit.text if revisit else ''
    time.sleep(0.06)

    # append sheet
    print(nickname, '/', content, '/', date, '/', revisit)
    list_sheet.append([nickname, content, date, revisit])
    time.sleep(0.06)

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from openpyxl import Workbook
from datetime import datetime
import time

# URL 설정
url = 'https://m.place.naver.com/restaurant/1085956231/review/visitor?entry=ple&reviewSort=recent'

# Webdriver 설정 (headless mode)
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')

# BS4 설정
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))

# Excel 파일 생성
now = datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit'])

try:
    # 크롤링 시작
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    res = driver.get(url)
    driver.implicitly_wait(30)
    
    # 페이지 아래로 스크롤
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    
    try:
        while True:
            # "더보기" 버튼 클릭 대기 및 클릭
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a'))
            ).click()
            time.sleep(0.4)
    except Exception as e:
        print('모든 리뷰 로드 완료 또는 버튼이 더 이상 없습니다:', e)

    # HTML 파싱
    time.sleep(25)
    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    reviews = bs.select('li.pui__X35jYm.EjjAW')

    for r in reviews:
        # 닉네임
        nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

        # 리뷰 내용
        content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

        # 날짜
        date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
        date = date_elements[0] if date_elements else 'N/A'

        # 재방문 여부
        revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
        revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

        # 예외 처리
        nickname = nickname.text if nickname else ''
        content = content.text if content else ''
        date = date.text if date else ''
        revisit = revisit.text if revisit else ''
        time.sleep(0.06)

        # Excel에 추가
        print(nickname, '/', content, '/', date, '/', revisit)
        list_sheet.append([nickname, content, date, revisit])
        time.sleep(0.06)

    # 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(f"오류 발생: {e}")
    # 임시 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

finally:
    # WebDriver 종료
    driver.quit()


모든 리뷰 로드 완료 또는 버튼이 더 이상 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00F338B3+24035]
	(No symbol) [0x00EBBC44]
	(No symbol) [0x00D9C2D3]
	(No symbol) [0x00DDDC86]
	(No symbol) [0x00DDDECB]
	(No symbol) [0x00E1B9D2]
	(No symbol) [0x00DFFED4]
	(No symbol) [0x00E1953F]
	(No symbol) [0x00DFFC26]
	(No symbol) [0x00DD218C]
	(No symbol) [0x00DD310D]
	GetHandleVerifier [0x011D96D3+2800643]
	GetHandleVerifier [0x0123428E+3172286]
	GetHandleVerifier [0x0122CEA2+3142610]
	GetHandleVerifier [0x00FD6C60+692624]
	(No symbol) [0x00EC4C5D]
	(No symbol) [0x00EC1968]
	(No symbol) [0x00EC1B00]
	(No symbol) [0x00EB3FB0]
	BaseThreadInitThunk [0x76C5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F3809E+286]
	RtlGetAppContainerNamedObjectPath [0x76F3806E+238]



### ----------------------------------------------------------

# -------------------------------------------------------------------

In [18]:
!pip install webdriver-manager

In [19]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [20]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 리뷰 추출
    rev = []  # 추출한 리뷰 저장
    for i in range(1, 11): # 더보기 누르지 않은 상태로 최대 10개
        try:  # 사진 없는 후기는 div 3번째에 텍스트 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
            rev.append(comment)
        except: # 사진 있는 후기는 div 4번째에 텍스트가 위치
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a').send_keys(Keys.ENTER)
            time.sleep(2)
            comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[3]/div[4]/div[1]/ul/li['+str(i)+']/div[4]/a/span').text  # 리뷰
            rev.append(comment)
    return rev

In [ ]:
data = []

In [ ]:
https://map.naver.com/p?c=15.00,0,0,0,dh

https://map.naver.com/p?c=15.00,0,0,0,dh

In [ ]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

for i in df['음식점명']:
    # 검색창에 입력하지 않고 직접 해당 숙소의 주소로 이동
    url = "https://map.naver.com/v5/search/서울 "+str(i)+'/place' 
    driver.get(url)
    time.sleep(8)

    
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(3) 
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 리뷰 탭으로 이동
        lists = soup.select('#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span')
        
        # 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
        if len(lists) > 2: 
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(3)
		
        # 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
        else: 
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(3)
        
        review = extract_review() # 리뷰 추출 함수 호출
        data.append(review)
        # print(review)
    
    except:
        data.append(' ')
        # print(' ')